## 1. Импорт библиотек и подготовка данных

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

### Загрузка и подготовка данных

In [ ]:
# Загрузка данных
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Названия классов
classes = ['футболка', 'брюки', 'свитер', 'платье', 'пальто', 
           'туфли', 'рубашка', 'кроссовки', 'сумка', 'ботинки']

print(f"Training set: {x_train.shape}")
print(f"Test set: {x_test.shape}")

In [ ]:
# Визуализация примеров
plt.figure(figsize=(10, 5))
for i in range(20):
    plt.subplot(2, 10, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(classes[y_train[i]], fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# Преобразование данных
x_train_flat = x_train.reshape(60000, 784)
x_test_flat = x_test.reshape(10000, 784)

# Нормализация
x_train_norm = x_train_flat / 255.0
x_test_norm = x_test_flat / 255.0

# One-hot encoding для меток
y_train_cat = utils.to_categorical(y_train, 10)
y_test_cat = utils.to_categorical(y_test, 10)

print(f"Preprocessed training data: {x_train_norm.shape}")
print(f"Preprocessed test data: {x_test_norm.shape}")
print(f"Labels shape: {y_train_cat.shape}")

## 2. Вспомогательные функции

In [ ]:
def create_model(layers_config, input_dim=784, output_dim=10):
    """
    Создает модель с заданной конфигурацией слоев
    
    Parameters:
    layers_config: list - список с количеством нейронов в каждом слое
    
    Returns:
    model: Sequential - скомпилированная модель
    """
    model = Sequential()
    
    # Первый слой
    model.add(Dense(layers_config[0], input_dim=input_dim, activation='relu'))
    
    # Скрытые слои (если есть)
    for neurons in layers_config[1:]:
        model.add(Dense(neurons, activation='relu'))
    
    # Выходной слой
    model.add(Dense(output_dim, activation='softmax'))
    
    # Компиляция
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    return model

def train_and_evaluate(model, x_train, y_train, x_test, y_test, 
                       epochs=20, batch_size=100, verbose=0):
    """
    Обучает и оценивает модель
    
    Returns:
    dict с результатами: history, test_loss, test_accuracy, params_count
    """
    # Обучение
    history = model.fit(x_train, y_train,
                       batch_size=batch_size,
                       epochs=epochs,
                       validation_split=0.2,
                       verbose=verbose)
    
    # Оценка на тестовых данных
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    
    # Подсчет параметров
    params_count = model.count_params()
    
    return {
        'history': history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'params_count': params_count,
        'train_accuracy': history.history['accuracy'][-1],
        'val_accuracy': history.history['val_accuracy'][-1]
    }

def plot_history(history, title='Training History'):
    """
    Визуализирует историю обучения
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy
    ax1.plot(history.history['accuracy'], label='Train')
    ax1.plot(history.history['val_accuracy'], label='Validation')
    ax1.set_title(f'{title} - Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Loss
    ax2.plot(history.history['loss'], label='Train')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_title(f'{title} - Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

print("Вспомогательные функции готовы!")

## 3. Эксперимент 1: Количество нейронов на входном слое

Тестируем: 400, 600, 800, 1200 нейронов

In [ ]:
# Эксперимент 1: Входной слой
input_layer_neurons = [400, 600, 800, 1200]
exp1_results = []

print("=" * 80)
print("ЭКСПЕРИМЕНТ 1: Количество нейронов на входном слое")
print("=" * 80)

for neurons in input_layer_neurons:
    print(f"\nОбучение модели с {neurons} нейронами на входном слое...")
    
    model = create_model([neurons])
    results = train_and_evaluate(model, x_train_norm, y_train_cat, 
                                x_test_norm, y_test_cat,
                                epochs=20, batch_size=100, verbose=1)
    
    exp1_results.append({
        'neurons': neurons,
        'test_accuracy': results['test_accuracy'],
        'val_accuracy': results['val_accuracy'],
        'params': results['params_count'],
        'history': results['history']
    })
    
    print(f"Точность на тесте: {results['test_accuracy']:.4f}")
    print(f"Точность на валидации: {results['val_accuracy']:.4f}")
    print(f"Параметры модели: {results['params_count']:,}")

print("\n" + "=" * 80)

In [ ]:
# Сравнение результатов Эксперимента 1
df_exp1 = pd.DataFrame([{
    'Нейроны': r['neurons'],
    'Точность (тест)': f"{r['test_accuracy']:.4f}",
    'Точность (вал)': f"{r['val_accuracy']:.4f}",
    'Параметры': f"{r['params']:,}"
} for r in exp1_results])

print("\nРезультаты Эксперимента 1:")
print(df_exp1.to_string(index=False))

# Находим лучшую модель
best_idx = np.argmax([r['test_accuracy'] for r in exp1_results])
print(f"\nЛучшая конфигурация: {exp1_results[best_idx]['neurons']} нейронов")
print(f"Точность: {exp1_results[best_idx]['test_accuracy']:.4f}")

In [ ]:
# Визуализация лучшей модели из Эксперимента 1
plot_history(exp1_results[best_idx]['history'], 
            f"Exp 1: {exp1_results[best_idx]['neurons']} neurons")

## 4. Эксперимент 2: Один скрытый слой

Добавляем один скрытый слой: 200, 300, 400, 600, 800 нейронов

In [ ]:
# Эксперимент 2: Один скрытый слой
hidden_layer_neurons = [200, 300, 400, 600, 800]
exp2_results = []

print("=" * 80)
print("ЭКСПЕРИМЕНТ 2: Добавление одного скрытого слоя")
print("=" * 80)

# Используем лучшую конфигурацию из Эксп. 1 для входного слоя
best_input_neurons = exp1_results[best_idx]['neurons']

for hidden_neurons in hidden_layer_neurons:
    print(f"\nОбучение модели: [{best_input_neurons}, {hidden_neurons}]...")
    
    model = create_model([best_input_neurons, hidden_neurons])
    results = train_and_evaluate(model, x_train_norm, y_train_cat, 
                                x_test_norm, y_test_cat,
                                epochs=20, batch_size=100, verbose=1)
    
    exp2_results.append({
        'config': f"{best_input_neurons}-{hidden_neurons}",
        'hidden_neurons': hidden_neurons,
        'test_accuracy': results['test_accuracy'],
        'val_accuracy': results['val_accuracy'],
        'params': results['params_count'],
        'history': results['history']
    })
    
    print(f"Точность на тесте: {results['test_accuracy']:.4f}")
    print(f"Точность на валидации: {results['val_accuracy']:.4f}")
    print(f"Параметры модели: {results['params_count']:,}")

print("\n" + "=" * 80)

In [ ]:
# Сравнение результатов Эксперимента 2
df_exp2 = pd.DataFrame([{
    'Конфигурация': r['config'],
    'Точность (тест)': f"{r['test_accuracy']:.4f}",
    'Точность (вал)': f"{r['val_accuracy']:.4f}",
    'Параметры': f"{r['params']:,}"
} for r in exp2_results])

print("\nРезультаты Эксперимента 2:")
print(df_exp2.to_string(index=False))

# Находим лучшую модель
best_idx2 = np.argmax([r['test_accuracy'] for r in exp2_results])
print(f"\nЛучшая конфигурация: {exp2_results[best_idx2]['config']}")
print(f"Точность: {exp2_results[best_idx2]['test_accuracy']:.4f}")

In [ ]:
# Визуализация лучшей модели из Эксперимента 2
plot_history(exp2_results[best_idx2]['history'], 
            f"Exp 2: {exp2_results[best_idx2]['config']}")

## 5. Эксперимент 3: Несколько скрытых слоев

Тестируем разные архитектуры с 2-3 скрытыми слоями

In [ ]:
# Эксперимент 3: Несколько скрытых слоев
multi_layer_configs = [
    [800, 400, 200],
    [800, 600, 400],
    [1200, 600, 300],
    [800, 400, 200, 100],
    [1200, 800, 400, 200]
]

exp3_results = []

print("=" * 80)
print("ЭКСПЕРИМЕНТ 3: Несколько скрытых слоев")
print("=" * 80)

for config in multi_layer_configs:
    config_str = '-'.join(map(str, config))
    print(f"\nОбучение модели: {config_str}...")
    
    model = create_model(config)
    results = train_and_evaluate(model, x_train_norm, y_train_cat, 
                                x_test_norm, y_test_cat,
                                epochs=20, batch_size=100, verbose=1)
    
    exp3_results.append({
        'config': config_str,
        'layers': len(config),
        'test_accuracy': results['test_accuracy'],
        'val_accuracy': results['val_accuracy'],
        'params': results['params_count'],
        'history': results['history']
    })
    
    print(f"Точность на тесте: {results['test_accuracy']:.4f}")
    print(f"Точность на валидации: {results['val_accuracy']:.4f}")
    print(f"Параметры модели: {results['params_count']:,}")

print("\n" + "=" * 80)

In [ ]:
# Сравнение результатов Эксперимента 3
df_exp3 = pd.DataFrame([{
    'Конфигурация': r['config'],
    'Слои': r['layers'],
    'Точность (тест)': f"{r['test_accuracy']:.4f}",
    'Точность (вал)': f"{r['val_accuracy']:.4f}",
    'Параметры': f"{r['params']:,}"
} for r in exp3_results])

print("\nРезультаты Эксперимента 3:")
print(df_exp3.to_string(index=False))

# Находим лучшую модель
best_idx3 = np.argmax([r['test_accuracy'] for r in exp3_results])
print(f"\nЛучшая конфигурация: {exp3_results[best_idx3]['config']}")
print(f"Точность: {exp3_results[best_idx3]['test_accuracy']:.4f}")

In [ ]:
# Визуализация лучшей модели из Эксперимента 3
plot_history(exp3_results[best_idx3]['history'], 
            f"Exp 3: {exp3_results[best_idx3]['config']}")

## 6. Эксперимент 4: Количество эпох

Тестируем: 10, 15, 20, 25, 30 эпох

In [ ]:
# Эксперимент 4: Количество эпох
epochs_list = [10, 15, 20, 25, 30]
exp4_results = []

print("=" * 80)
print("ЭКСПЕРИМЕНТ 4: Количество эпох")
print("=" * 80)

# Используем лучшую архитектуру из предыдущих экспериментов
best_overall_idx = best_idx3
best_config = multi_layer_configs[best_overall_idx]

for epochs in epochs_list:
    print(f"\nОбучение модели с {epochs} эпохами...")
    
    model = create_model(best_config)
    results = train_and_evaluate(model, x_train_norm, y_train_cat, 
                                x_test_norm, y_test_cat,
                                epochs=epochs, batch_size=100, verbose=1)
    
    exp4_results.append({
        'epochs': epochs,
        'test_accuracy': results['test_accuracy'],
        'val_accuracy': results['val_accuracy'],
        'train_accuracy': results['train_accuracy'],
        'history': results['history']
    })
    
    print(f"Точность на тесте: {results['test_accuracy']:.4f}")
    print(f"Точность на валидации: {results['val_accuracy']:.4f}")
    print(f"Точность на обучении: {results['train_accuracy']:.4f}")

print("\n" + "=" * 80)

In [ ]:
# Сравнение результатов Эксперимента 4
df_exp4 = pd.DataFrame([{
    'Эпохи': r['epochs'],
    'Точность (тест)': f"{r['test_accuracy']:.4f}",
    'Точность (вал)': f"{r['val_accuracy']:.4f}",
    'Точность (обуч)': f"{r['train_accuracy']:.4f}",
    'Переобучение': f"{r['train_accuracy'] - r['val_accuracy']:.4f}"
} for r in exp4_results])

print("\nРезультаты Эксперимента 4:")
print(df_exp4.to_string(index=False))

# Находим оптимальное количество эпох
best_idx4 = np.argmax([r['test_accuracy'] for r in exp4_results])
print(f"\nОптимальное количество эпох: {exp4_results[best_idx4]['epochs']}")
print(f"Точность: {exp4_results[best_idx4]['test_accuracy']:.4f}")

In [ ]:
# Визуализация всех вариантов с разным количеством эпох
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, result in enumerate(exp4_results):
    ax = axes[idx]
    ax.plot(result['history'].history['accuracy'], label='Train')
    ax.plot(result['history'].history['val_accuracy'], label='Validation')
    ax.set_title(f"{result['epochs']} epochs")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.legend()
    ax.grid(True)

# Скрываем лишний subplot
axes[-1].axis('off')

plt.tight_layout()
plt.show()

## 7. Эксперимент 5: Размер batch

Тестируем: 10, 50, 100, 200, 500

In [ ]:
# Эксперимент 5: Размер batch
batch_sizes = [10, 50, 100, 200, 500]
exp5_results = []

print("=" * 80)
print("ЭКСПЕРИМЕНТ 5: Размер batch")
print("=" * 80)

# Используем лучшую конфигурацию и оптимальное количество эпох
best_epochs = exp4_results[best_idx4]['epochs']

import time

for batch_size in batch_sizes:
    print(f"\nОбучение модели с batch_size={batch_size}...")
    
    model = create_model(best_config)
    
    start_time = time.time()
    results = train_and_evaluate(model, x_train_norm, y_train_cat, 
                                x_test_norm, y_test_cat,
                                epochs=best_epochs, batch_size=batch_size, verbose=1)
    training_time = time.time() - start_time
    
    exp5_results.append({
        'batch_size': batch_size,
        'test_accuracy': results['test_accuracy'],
        'val_accuracy': results['val_accuracy'],
        'training_time': training_time,
        'history': results['history']
    })
    
    print(f"Точность на тесте: {results['test_accuracy']:.4f}")
    print(f"Время обучения: {training_time:.2f} сек")

print("\n" + "=" * 80)

In [ ]:
# Сравнение результатов Эксперимента 5
df_exp5 = pd.DataFrame([{
    'Batch Size': r['batch_size'],
    'Точность (тест)': f"{r['test_accuracy']:.4f}",
    'Точность (вал)': f"{r['val_accuracy']:.4f}",
    'Время (сек)': f"{r['training_time']:.2f}"
} for r in exp5_results])

print("\nРезультаты Эксперимента 5:")
print(df_exp5.to_string(index=False))

# Находим лучший batch_size
best_idx5 = np.argmax([r['test_accuracy'] for r in exp5_results])
print(f"\nОптимальный batch_size: {exp5_results[best_idx5]['batch_size']}")
print(f"Точность: {exp5_results[best_idx5]['test_accuracy']:.4f}")

## 8. Анализ переобучения для лучшей модели

In [ ]:
# Обучаем лучшую модель с большим количеством эпох для анализа переобучения
print("=" * 80)
print("АНАЛИЗ ПЕРЕОБУЧЕНИЯ")
print("=" * 80)

best_batch_size = exp5_results[best_idx5]['batch_size']

print(f"\nЛучшая конфигурация:")
print(f"Архитектура: {'-'.join(map(str, best_config))}")
print(f"Эпохи: {best_epochs}")
print(f"Batch size: {best_batch_size}")

# Создаем финальную модель
final_model = create_model(best_config)

# Обучаем с расширенным количеством эпох для проверки переобучения
final_history = final_model.fit(x_train_norm, y_train_cat,
                               batch_size=best_batch_size,
                               epochs=50,  # Увеличиваем для анализа
                               validation_split=0.2,
                               verbose=1)

# Оценка
test_loss, test_accuracy = final_model.evaluate(x_test_norm, y_test_cat, verbose=0)

print(f"\nФинальная точность на тесте: {test_accuracy:.4f}")
print(f"Финальная точность на валидации: {final_history.history['val_accuracy'][-1]:.4f}")
print(f"Финальная точность на обучении: {final_history.history['accuracy'][-1]:.4f}")

In [ ]:
# Визуализация переобучения
plot_history(final_history, "Final Model - Overfitting Analysis (50 epochs)")

# Проверка на переобучение
train_acc = final_history.history['accuracy'][-1]
val_acc = final_history.history['val_accuracy'][-1]
overfitting_gap = train_acc - val_acc

print(f"\nРазница между train и validation accuracy: {overfitting_gap:.4f}")
if overfitting_gap > 0.05:
    print("⚠️  Обнаружено переобучение! Разница > 5%")
else:
    print("✓ Переобучение отсутствует или минимально")

In [ ]:
# Сводка модели
print("\nАрхитектура финальной модели:")
final_model.summary()

## 9. Keras Tuner - Автоматический подбор гиперпараметров

In [ ]:
# Установка Keras Tuner (если еще не установлен)
import subprocess
import sys

try:
    import keras_tuner as kt
    print(f"Keras Tuner уже установлен, версия: {kt.__version__}")
except ImportError:
    print("Установка Keras Tuner...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "keras-tuner", "-q"])
    import keras_tuner as kt
    print(f"Keras Tuner установлен, версия: {kt.__version__}")

In [ ]:
# Определение модели для Keras Tuner
def build_tuner_model(hp):
    """
    Функция построения модели для Keras Tuner
    """
    model = Sequential()
    
    # Входной слой с настраиваемым количеством нейронов
    model.add(Dense(
        units=hp.Int('input_units', min_value=400, max_value=1200, step=200),
        activation='relu',
        input_dim=784
    ))
    
    # Настраиваемое количество скрытых слоев (1-3)
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=200, max_value=800, step=200),
            activation='relu'
        ))
        
        # Опциональный Dropout
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(hp.Float(f'dropout_rate_{i}', 0.1, 0.5, step=0.1)))
    
    # Выходной слой
    model.add(Dense(10, activation='softmax'))
    
    # Настраиваемый learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

print("Функция построения модели для Keras Tuner готова!")

In [ ]:
# Создание и настройка тюнера
print("=" * 80)
print("KERAS TUNER - Автоматический подбор гиперпараметров")
print("=" * 80)

tuner = kt.Hyperband(
    build_tuner_model,
    objective='val_accuracy',
    max_epochs=30,
    factor=3,
    hyperband_iterations=2,
    directory='kt_dir',
    project_name='fashion_mnist_lab4',
    overwrite=True
)

print("\nПространство поиска:")
tuner.search_space_summary()

In [ ]:
# Запуск поиска
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

print("\nЗапуск поиска оптимальных гиперпараметров...")
print("Это может занять некоторое время...\n")

tuner.search(
    x_train_norm, y_train_cat,
    epochs=30,
    validation_split=0.2,
    callbacks=[stop_early],
    verbose=1
)

print("\n" + "=" * 80)
print("Поиск завершен!")
print("=" * 80)

In [ ]:
# Получение лучших гиперпараметров
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\nЛучшие найденные гиперпараметры:")
print(f"Входной слой: {best_hps.get('input_units')} нейронов")
print(f"Количество скрытых слоев: {best_hps.get('num_layers')}")

for i in range(best_hps.get('num_layers')):
    units = best_hps.get(f'units_{i}')
    has_dropout = best_hps.get(f'dropout_{i}')
    print(f"  Слой {i+1}: {units} нейронов", end="")
    if has_dropout:
        dropout_rate = best_hps.get(f'dropout_rate_{i}')
        print(f", Dropout={dropout_rate}")
    else:
        print()

print(f"Learning rate: {best_hps.get('learning_rate')}")

In [ ]:
# Обучение модели с лучшими гиперпараметрами
print("\nОбучение модели с найденными гиперпараметрами...\n")

tuned_model = tuner.hypermodel.build(best_hps)

tuned_history = tuned_model.fit(
    x_train_norm, y_train_cat,
    epochs=30,
    validation_split=0.2,
    callbacks=[stop_early],
    verbose=1
)

# Оценка на тестовых данных
tuned_test_loss, tuned_test_accuracy = tuned_model.evaluate(x_test_norm, y_test_cat, verbose=0)

print(f"\nТочность модели Keras Tuner на тесте: {tuned_test_accuracy:.4f}")

In [ ]:
# Визуализация модели Keras Tuner
plot_history(tuned_history, "Keras Tuner - Best Model")

print("\nАрхитектура модели Keras Tuner:")
tuned_model.summary()

## 10. Сравнение результатов: Ручной vs Автоматический подбор

In [ ]:
# Итоговое сравнение
print("=" * 80)
print("ИТОГОВОЕ СРАВНЕНИЕ РЕЗУЛЬТАТОВ")
print("=" * 80)

# Подсчет параметров
manual_params = final_model.count_params()
tuned_params = tuned_model.count_params()

comparison_data = {
    'Метод': ['Ручной подбор', 'Keras Tuner'],
    'Архитектура': [
        '-'.join(map(str, best_config)),
        f"{best_hps.get('input_units')}-" + '-'.join([str(best_hps.get(f'units_{i}')) for i in range(best_hps.get('num_layers'))])
    ],
    'Точность (тест)': [
        f"{test_accuracy:.4f}",
        f"{tuned_test_accuracy:.4f}"
    ],
    'Точность (вал)': [
        f"{final_history.history['val_accuracy'][-1]:.4f}",
        f"{tuned_history.history['val_accuracy'][-1]:.4f}"
    ],
    'Параметры': [
        f"{manual_params:,}",
        f"{tuned_params:,}"
    ],
    'Эпохи': [
        str(best_epochs),
        str(len(tuned_history.history['accuracy']))
    ],
    'Batch Size': [
        str(best_batch_size),
        "N/A"
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n", df_comparison.to_string(index=False))

print("\n" + "=" * 80)

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# График точности
methods = ['Ручной\nподбор', 'Keras\nTuner']
accuracies = [test_accuracy, tuned_test_accuracy]
colors = ['#3498db', '#e74c3c']

axes[0].bar(methods, accuracies, color=colors, alpha=0.7)
axes[0].set_ylabel('Точность на тесте')
axes[0].set_title('Сравнение точности')
axes[0].set_ylim([0.8, 0.95])
axes[0].grid(True, alpha=0.3)

# Добавляем значения на столбцы
for i, (method, acc) in enumerate(zip(methods, accuracies)):
    axes[0].text(i, acc + 0.005, f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')

# График размера модели
params = [manual_params, tuned_params]

axes[1].bar(methods, params, color=colors, alpha=0.7)
axes[1].set_ylabel('Количество параметров')
axes[1].set_title('Сравнение размера модели')
axes[1].grid(True, alpha=0.3)

# Добавляем значения на столбцы
for i, (method, param) in enumerate(zip(methods, params)):
    axes[1].text(i, param + max(params)*0.02, f'{param:,}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 11. Выводы

In [ ]:
print("=" * 80)
print("ВЫВОДЫ")
print("=" * 80)

print("\n1. ВЛИЯНИЕ КОЛИЧЕСТВА НЕЙРОНОВ НА ВХОДНОМ СЛОЕ:")
print(f"   - Протестировано: {input_layer_neurons}")
print(f"   - Лучший результат: {exp1_results[best_idx]['neurons']} нейронов")
print(f"   - Точность: {exp1_results[best_idx]['test_accuracy']:.4f}")
print("   - Вывод: Увеличение нейронов улучшает точность до определенного предела")

print("\n2. ВЛИЯНИЕ СКРЫТЫХ СЛОЕВ:")
print(f"   - Добавление 1 скрытого слоя улучшило результат")
print(f"   - Лучшая конфигурация с 1 слоем: {exp2_results[best_idx2]['config']}")
print(f"   - Точность: {exp2_results[best_idx2]['test_accuracy']:.4f}")

print("\n3. МНОЖЕСТВЕННЫЕ СКРЫТЫЕ СЛОИ:")
print(f"   - Лучшая архитектура: {exp3_results[best_idx3]['config']}")
print(f"   - Точность: {exp3_results[best_idx3]['test_accuracy']:.4f}")
print("   - Вывод: Глубокие сети могут давать лучшие результаты, но требуют больше параметров")

print("\n4. КОЛИЧЕСТВО ЭПОХ:")
print(f"   - Оптимальное количество: {exp4_results[best_idx4]['epochs']} эпох")
print(f"   - Переобучение начинается после {best_epochs} эпох")
print(f"   - Разница train/val: {overfitting_gap:.4f}")

print("\n5. РАЗМЕР BATCH:")
print(f"   - Оптимальный batch_size: {exp5_results[best_idx5]['batch_size']}")
print(f"   - Меньший batch дает лучшую точность, но медленнее обучается")
print(f"   - Больший batch ускоряет обучение, но может снизить точность")

print("\n6. СРАВНЕНИЕ РУЧНОГО И АВТОМАТИЧЕСКОГО ПОДБОРА:")
if test_accuracy > tuned_test_accuracy:
    winner = "Ручной подбор"
    diff = test_accuracy - tuned_test_accuracy
else:
    winner = "Keras Tuner"
    diff = tuned_test_accuracy - test_accuracy

print(f"   - Победитель: {winner}")
print(f"   - Разница в точности: {diff:.4f}")
print(f"   - Ручной подбор: {test_accuracy:.4f} ({manual_params:,} параметров)")
print(f"   - Keras Tuner: {tuned_test_accuracy:.4f} ({tuned_params:,} параметров)")

if manual_params < tuned_params:
    print(f"   - Ручная модель компактнее на {((tuned_params - manual_params) / tuned_params * 100):.1f}%")
else:
    print(f"   - Keras Tuner модель компактнее на {((manual_params - tuned_params) / manual_params * 100):.1f}%")

print("\n7. ОБЩИЕ ВЫВОДЫ:")
print("   - Систематический ручной подбор может конкурировать с автоматическим")
print("   - Keras Tuner экономит время на эксперименты")
print("   - Важно контролировать переобучение независимо от метода подбора")
print("   - Оптимальная архитектура зависит от баланса точность/размер/время обучения")

print("\n" + "=" * 80)

## 12. Сохранение лучших моделей

In [ ]:
# Сохранение моделей
final_model.save('best_manual_model.h5')
tuned_model.save('best_tuned_model.h5')

print("Модели сохранены:")
print("- best_manual_model.h5 (ручной подбор)")
print("- best_tuned_model.h5 (Keras Tuner)")